In [1]:
from __future__ import division
import math
import os
import mne
from mne import io
from mne.event import define_target_events
import numpy as np
import h5py
import matplotlib.pyplot as plt
from PIL import Image
import glob

In [17]:
subjects_arr=["D:/001/arti/eegArtiBablu/eyeBlink/*","D:/001/arti/eegArti11to15/eyeBlink/*", "D:/001/arti/eegArti16to20/eyeBlink/*",
              "D:/001/arti/eegArtiAshutosh/eyeBlink/*"]#"D:/001/arti/eegArti6to10/eyeBlink/*"

In [18]:
arr= glob.glob("D:/001/arti/eegArtiAshutosh/eyeBlink/*")
print (arr)

['D:/001/arti/eegArtiAshutosh/eyeBlink\\eyeBlink_subject_21_2019.08.13_18.05.15.hdf5', 'D:/001/arti/eegArtiAshutosh/eyeBlink\\eyeBlink_subject_22_2019.08.13_18.06.44.hdf5', 'D:/001/arti/eegArtiAshutosh/eyeBlink\\eyeBlink_subject_23_2019.08.13_18.08.12.hdf5', 'D:/001/arti/eegArtiAshutosh/eyeBlink\\eyeBlink_subject_24_2019.08.13_18.09.35.hdf5', 'D:/001/arti/eegArtiAshutosh/eyeBlink\\eyeBlink_subject_25_2019.08.13_18.10.57.hdf5']


In [19]:
name= arr[0][arr[0].find('_'): arr[0].find('_')+11]
print (name)


_subject_21


In [20]:
montage= mne.channels.read_montage('63electrodes', ch_names=None, path="D:/001/arti", unit='auto', transform=False)
ch_n=['Fp1', 'Fpz', 'Fp2', 'AF7', 'AF3','AF4', 'AF8', 'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6',
     'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FT8', 'T7', 'C5', 'C3', 'C1', 'Cz', 
     'C2', 'C4', 'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'TP8', 'P7', 'P5',
     'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'O1', 'Oz', 'O2', 'F9',
     'F10', 'A1']
info= mne.create_info(ch_names= ch_n, sfreq= 2400, ch_types='eeg', montage=montage, verbose=None)

In [21]:
for subjects in subjects_arr:
    arr= glob.glob(subjects)
    for files in arr:
        eeg= h5py.File(files, mode='r')
        samples= np.array(eeg['RawData']['Samples'])
        samples= samples.T
        raw= mne.io.RawArray(data= samples, info= info)
        raw.filter(1., 100, n_jobs=1, fir_design='firwin')
        sampletime = np.array(eeg['AsynchronData']["Time"])
        raw.info['bads'] = ['Fp1', 'A1']
        raw.interpolate_bads(reset_bads=False)
        for i in range(1, len(sampletime)-1):
            current= i-1
            prev= i-2
            eve= sampletime[current]
            eve= np.hstack((eve, np.array(prev)))
            eve= np.hstack((eve, np.array(current)))
            if i==1:
                events= eve
            else:
                events= np.vstack((events,eve))

        for ids in range(0, len(sampletime)-2):
            epochs = mne.Epochs(raw, events, event_id= ids, tmin= -2.5, tmax= 2.5, preload=True)#time= 2.5 for eye blink, 5 for others
            epochs_resampled = epochs.copy().resample(512, npad='auto')#512 for eye blink 256 for others
            save_name='D:/001/arti/eyeBlink/eyeBlink'+ files[files.find('_'): files.find('_')+17]+"_epoch_"+str(ids)+"_epo.fif"
            epochs_resampled.save(save_name, overwrite= True)

Creating RawArray with float64 data, n_channels=63, n_times=351360
    Range : 0 ... 351359 =      0.000 ...   146.400 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 7921 samples (3.300 sec)

Computing interpolation matrix from 61 sensor positions
Interpolating 2 sensors
1 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1 events and 12001 original time points ...
0 bad epochs dropped
1 matching events foun

Loading data for 1 events and 12001 original time points ...
0 bad epochs dropped
Creating RawArray with float64 data, n_channels=63, n_times=220800
    Range : 0 ... 220799 =      0.000 ...    92.000 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 7921 samples (3.300 sec)

Computing interpolation matrix from 61 sensor positions
Interpolating 2 sensors
1 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1 eve

Not setting metadata
0 projection items activated
Loading data for 1 events and 12001 original time points ...
0 bad epochs dropped
Creating RawArray with float64 data, n_channels=63, n_times=186864
    Range : 0 ... 186863 =      0.000 ...    77.860 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 7921 samples (3.300 sec)

Computing interpolation matrix from 61 sensor positions
Interpolating 2 sensors
1 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0

0 bad epochs dropped
1 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1 events and 12001 original time points ...
0 bad epochs dropped
Creating RawArray with float64 data, n_channels=63, n_times=190512
    Range : 0 ... 190511 =      0.000 ...    79.380 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 7921 samples (3.300 sec)

Computing interpolation matrix from 61 sensor positions
Interpolating 2 sensors
1

Loading data for 1 events and 12001 original time points ...
0 bad epochs dropped
1 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1 events and 12001 original time points ...
0 bad epochs dropped
Creating RawArray with float64 data, n_channels=63, n_times=178032
    Range : 0 ... 178031 =      0.000 ...    74.180 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 7921 samples (3.300 sec)

Computing interpolat

Not setting metadata
0 projection items activated
Loading data for 1 events and 12001 original time points ...
0 bad epochs dropped
1 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1 events and 12001 original time points ...
0 bad epochs dropped
Creating RawArray with float64 data, n_channels=63, n_times=198144
    Range : 0 ... 198143 =      0.000 ...    82.560 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter leng

0 bad epochs dropped
1 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1 events and 12001 original time points ...
0 bad epochs dropped
1 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1 events and 12001 original time points ...
0 bad epochs dropped
Creating RawArray with float64 data, n_channels=63, n_times=184080
    Range : 0 ... 184079 =      0.000 ...    76.700 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 100.00 Hz
-